Fernanda Macedo de Sousa - 17/0010058

Mariana Alencar do Vale - 16/0014522  


O projeto visa usar e testar variações de uma rede CNN (Rede Neuronal Convolucional, ou *Convolutional Neural Network*) implementada no *framework* Keras para reconhecimento de dígitos manuscritos, a partir da base do MNIST.

---




# Importando o Dataset

In [ ]:

from keras.datasets.mnist import load_data

Carregando os dígitos de treino e os dígitos de teste, em formato de tuplas com seus respectivos rótulos.

In [ ]:
(train_digits, train_labels), (test_digits, test_labels) = load_data()

11493376/11490434 [==============================] - 0s 0us/step


# Pré processamento dos dados do Dataset

In [ ]:
import numpy as np

image_height = train_digits.shape[1]  
image_width = train_digits.shape[2]
num_channels = 1  

train_data = np.reshape(train_digits, (train_digits.shape[0], image_height, image_width, num_channels))
test_data = np.reshape(test_digits, (test_digits.shape[0],image_height, image_width, num_channels))

train_data = train_data.astype('float32') / 255.
test_data = test_data.astype('float32') / 255.

from keras.utils import to_categorical
num_classes = 10
train_labels_cat = to_categorical(train_labels,num_classes)
test_labels_cat = to_categorical(test_labels,num_classes)
train_labels_cat.shape, test_labels_cat.shape

((60000, 10), (10000, 10))

Os dados foram normalizados. O Dataset foi dividido em 60.000 imagens de treino e 10.000 imagens de teste. 

# Separação de uma parte dos dados de treino do Dataset para validação

In [ ]:
for _ in range(5): 
    indexes = np.random.permutation(len(train_data))

train_data = train_data[indexes]
train_labels_cat = train_labels_cat[indexes]

val_perc = 0.10
val_count = int(val_perc * len(train_data))

val_data = train_data[:val_count,:]
val_labels_cat = train_labels_cat[:val_count,:]

train_data2 = train_data[val_count:,:]
train_labels_cat2 = train_labels_cat[val_count:,:]

Os dados foram embaralhados para garantir que não existe um padrão. Então foi definido que os primeiros 10% dos dados serão utilizados para a validação do modelo. Os outros 90% serão utilizados para treino.

# Construção da CNN

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def build_model():
    model = Sequential()
    # add Convolutional layers
    model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same',
                     input_shape=(image_height, image_width, num_channels)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))    
    model.add(Flatten())
    # Densely connected layers
    model.add(Dense(128, activation='relu'))
    # output layer
    model.add(Dense(num_classes, activation='softmax'))
    # compile with adam optimizer & categorical_crossentropy loss function
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_model()
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 576)               0

Inicialmente, nosso modelo CNN consiste em 3 camadas convolucionais, com uma camada *MaxPooling2D* com *pool_size=(2,2)* imediatamente após cada uma delas.

# Treino da CNN

In [ ]:
results = model.fit(train_data2, train_labels_cat2, 
                    epochs=15, batch_size=64,
                    validation_data=(val_data, val_labels_cat))


Epoch 1/15
844/844 [==============================] - 74s 87ms/step - loss: 0.1862 - accuracy: 0.9412 - val_loss: 0.0595 - val_accuracy: 0.9815
Epoch 2/15
844/844 [==============================] - 74s 87ms/step - loss: 0.0489 - accuracy: 0.9851 - val_loss: 0.0409 - val_accuracy: 0.9878
Epoch 3/15
844/844 [==============================] - 74s 87ms/step - loss: 0.0355 - accuracy: 0.9886 - val_loss: 0.0410 - val_accuracy: 0.9877
Epoch 4/15
844/844 [==============================] - 74s 87ms/step - loss: 0.0262 - accuracy: 0.9917 - val_loss: 0.0401 - val_accuracy: 0.9878
Epoch 5/15
844/844 [==============================] - 73s 87ms/step - loss: 0.0205 - accuracy: 0.9932 - val_loss: 0.0345 - val_accuracy: 0.9905
Epoch 6/15
844/844 [==============================] - 73s 87ms/step - loss: 0.0173 - accuracy: 0.9943 - val_loss: 0.0430 - val_accuracy: 0.9863
Epoch 7/15
844/844 [==============================] - 73s 87ms/step - loss: 0.0145 - accuracy: 0.9955 - val_loss: 0.0428 - val_accuracy:

# Avaliação de resultados

In [ ]:
test_loss, test_accuracy = model.evaluate(test_data, test_labels_cat, batch_size=64)
print('Test loss: %.4f accuracy: %.4f' % (test_loss, test_accuracy))

157/157 [==============================] - 4s 25ms/step - loss: 0.0345 - accuracy: 0.9925
Test loss: 0.0345 accuracy: 0.9925


A CNN com 3 camadas convolucionais, dados normalizados, com *MaxPooling* após cada camada convolucional, usando 10% dos dados de treinamento para validação e 90% restantes para treinamento, obteve 99.1% de acurácia. 

# Avaliação de resultados da CNN para esse Dataset sem normalizar os dados 

Analogamente, seguimos os passos de importação do Dataset em tuplas de treino e teste, pré processamento dos dados (embora sem a normalização) e separamos os dados de treino com 10% para validação e 90% para treino, assim como o modelo anterior. Da mesma forma, construimos a CNN e treinamos, a fim de avaliar a diferença dos resultados com e sem uma normalização dos dados. 

In [ ]:
(train_digits, train_labels), (test_digits, test_labels) = load_data()

In [ ]:
import numpy as np

image_height = train_digits.shape[1]  
image_width = train_digits.shape[2]
num_channels = 1  

train_data = np.reshape(train_digits, (train_digits.shape[0], image_height, image_width, num_channels))
test_data = np.reshape(test_digits, (test_digits.shape[0],image_height, image_width, num_channels))

from keras.utils import to_categorical
num_classes = 10
train_labels_cat = to_categorical(train_labels,num_classes)
test_labels_cat = to_categorical(test_labels,num_classes)
train_labels_cat.shape, test_labels_cat.shape

((60000, 10), (10000, 10))

In [ ]:
for _ in range(5): 
    indexes = np.random.permutation(len(train_data))

train_data = train_data[indexes]
train_labels_cat = train_labels_cat[indexes]

val_perc = 0.10
val_count = int(val_perc * len(train_data))

val_data = train_data[:val_count,:]
val_labels_cat = train_labels_cat[:val_count,:]

train_data2 = train_data[val_count:,:]
train_labels_cat2 = train_labels_cat[val_count:,:]


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def build_model():
    model = Sequential()
    # add Convolutional layers
    model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same',
                     input_shape=(image_height, image_width, num_channels)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))    
    model.add(Flatten())
    # Densely connected layers
    model.add(Dense(128, activation='relu'))
    # output layer
    model.add(Dense(num_classes, activation='softmax'))
    # compile with adam optimizer & categorical_crossentropy loss function
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_model()
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 3, 3, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)              

In [ ]:
results = model.fit(train_data2, train_labels_cat2, 
                    epochs=15, batch_size=64,
                    validation_data=(val_data, val_labels_cat))

Epoch 1/15
844/844 [==============================] - 73s 86ms/step - loss: 0.3718 - accuracy: 0.9306 - val_loss: 0.0806 - val_accuracy: 0.9753
Epoch 2/15
844/844 [==============================] - 73s 87ms/step - loss: 0.0629 - accuracy: 0.9809 - val_loss: 0.0583 - val_accuracy: 0.9818
Epoch 3/15
844/844 [==============================] - 73s 86ms/step - loss: 0.0462 - accuracy: 0.9851 - val_loss: 0.0594 - val_accuracy: 0.9835
Epoch 4/15
844/844 [==============================] - 73s 87ms/step - loss: 0.0405 - accuracy: 0.9871 - val_loss: 0.0524 - val_accuracy: 0.9860
Epoch 5/15
844/844 [==============================] - 73s 87ms/step - loss: 0.0332 - accuracy: 0.9894 - val_loss: 0.0499 - val_accuracy: 0.9843
Epoch 6/15
844/844 [==============================] - 73s 87ms/step - loss: 0.0329 - accuracy: 0.9898 - val_loss: 0.0683 - val_accuracy: 0.9823
Epoch 7/15
844/844 [==============================] - 74s 88ms/step - loss: 0.0265 - accuracy: 0.9914 - val_loss: 0.0554 - val_accuracy:

In [ ]:
test_loss, test_accuracy = model.evaluate(test_data, test_labels_cat, batch_size=64)
print('Test loss: %.4f accuracy: %.4f' % (test_loss, test_accuracy))

157/157 [==============================] - 4s 25ms/step - loss: 0.0448 - accuracy: 0.9902
Test loss: 0.0448 accuracy: 0.9902


Ao repetir os mesmos passos com os dados não normalizados, obtivemos uma acurácia de 98.46%, ou seja, 0.54% menor do que o modelo CNN que utiliza os dados normalizados (implementado anteriormente). 

# Usando somente uma camada convolucional

Nesta etapa, deixaremos apenas uma camada convolucional e avaliaremos o desempenho. Será utilizado 90% de dados para treinamento e 10% validação.

In [ ]:
(train_digits, train_labels), (test_digits, test_labels) = load_data()

import numpy as np

image_height = train_digits.shape[1]  
image_width = train_digits.shape[2]
num_channels = 1  

train_data = np.reshape(train_digits, (train_digits.shape[0], image_height, image_width, num_channels))
test_data = np.reshape(test_digits, (test_digits.shape[0],image_height, image_width, num_channels))

train_data = train_data.astype('float32') / 255.
test_data = test_data.astype('float32') / 255.

from keras.utils import to_categorical
num_classes = 10
train_labels_cat = to_categorical(train_labels,num_classes)
test_labels_cat = to_categorical(test_labels,num_classes)
train_labels_cat.shape, test_labels_cat.shape

for _ in range(5): 
    indexes = np.random.permutation(len(train_data))

train_data = train_data[indexes]
train_labels_cat = train_labels_cat[indexes]

val_perc = 0.1
val_count = int(val_perc * len(train_data))

val_data = train_data[:val_count,:]
val_labels_cat = train_labels_cat[:val_count,:]

train_data2 = train_data[val_count:,:]
train_labels_cat2 = train_labels_cat[val_count:,:]

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def build_model():
    model = Sequential()
    # add Convolutional layers
    model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same',
                     input_shape=(image_height, image_width, num_channels)))
    model.add(MaxPooling2D(pool_size=(2,2))) 
    model.add(Flatten())
    # Densely connected layers
    model.add(Dense(128, activation='relu'))
    # output layer
    model.add(Dense(num_classes, activation='softmax'))
    # compile with adam optimizer & categorical_crossentropy loss function
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_model()
print(model.summary())

results = model.fit(train_data2, train_labels_cat2, 
                    epochs=15, batch_size=64,
                    validation_data=(val_data, val_labels_cat))

test_loss, test_accuracy = model.evaluate(test_data, test_labels_cat, batch_size=64)
print('Test loss: %.4f accuracy: %.4f' % (test_loss, test_accuracy))

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               802944    
_________________________________________________________________
dense_9 (Dense)              (None, 10)                1290      
Total params: 804,554
Trainable params: 804,554
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/15
844/844 [==============================] - 28s 33ms/step - loss: 0.1931 - accuracy: 0.9441 - val_l

# Usando duas camadas convolucionais

Nesta etapa, iremos refazer todo o experimento, devolvendo uma das camadas convolucionais.

In [ ]:
(train_digits, train_labels), (test_digits, test_labels) = load_data()

import numpy as np

image_height = train_digits.shape[1]  
image_width = train_digits.shape[2]
num_channels = 1  

train_data = np.reshape(train_digits, (train_digits.shape[0], image_height, image_width, num_channels))
test_data = np.reshape(test_digits, (test_digits.shape[0],image_height, image_width, num_channels))

train_data = train_data.astype('float32') / 255.
test_data = test_data.astype('float32') / 255.

from keras.utils import to_categorical
num_classes = 10
train_labels_cat = to_categorical(train_labels,num_classes)
test_labels_cat = to_categorical(test_labels,num_classes)
train_labels_cat.shape, test_labels_cat.shape

for _ in range(5): 
    indexes = np.random.permutation(len(train_data))

train_data = train_data[indexes]
train_labels_cat = train_labels_cat[indexes]

val_perc = 0.1
val_count = int(val_perc * len(train_data))

val_data = train_data[:val_count,:]
val_labels_cat = train_labels_cat[:val_count,:]

train_data2 = train_data[val_count:,:]
train_labels_cat2 = train_labels_cat[val_count:,:]

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def build_model():
    model = Sequential()
    # add Convolutional layers
    model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same',
                     input_shape=(image_height, image_width, num_channels)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2))) 
    model.add(Flatten())
    # Densely connected layers
    model.add(Dense(128, activation='relu'))
    # output layer
    model.add(Dense(num_classes, activation='softmax'))
    # compile with adam optimizer & categorical_crossentropy loss function
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_model()
print(model.summary())

results = model.fit(train_data2, train_labels_cat2, 
                    epochs=15, batch_size=64,
                    validation_data=(val_data, val_labels_cat))

test_loss, test_accuracy = model.evaluate(test_data, test_labels_cat, batch_size=64)
print('Test loss: %.4f accuracy: %.4f' % (test_loss, test_accuracy))

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               401536    
_________________________________________________________________
dense_7 (Dense)              (None, 10)               

# Para 30% validação e 70% treinamento


Nesta etapa, foi refeito todo o processo (normalizado) mudando apenas a porcentagem de treinamento e validação com três camadas convolucionais.

In [ ]:
(train_digits, train_labels), (test_digits, test_labels) = load_data()

In [ ]:
import numpy as np

image_height = train_digits.shape[1]  
image_width = train_digits.shape[2]
num_channels = 1  

train_data = np.reshape(train_digits, (train_digits.shape[0], image_height, image_width, num_channels))
test_data = np.reshape(test_digits, (test_digits.shape[0],image_height, image_width, num_channels))

train_data = train_data.astype('float32') / 255.
test_data = test_data.astype('float32') / 255.

from keras.utils import to_categorical
num_classes = 10
train_labels_cat = to_categorical(train_labels,num_classes)
test_labels_cat = to_categorical(test_labels,num_classes)
train_labels_cat.shape, test_labels_cat.shape

((60000, 10), (10000, 10))

In [ ]:
for _ in range(5): 
    indexes = np.random.permutation(len(train_data))

train_data = train_data[indexes]
train_labels_cat = train_labels_cat[indexes]

val_perc = 0.3
val_count = int(val_perc * len(train_data))

val_data = train_data[:val_count,:]
val_labels_cat = train_labels_cat[:val_count,:]

train_data2 = train_data[val_count:,:]
train_labels_cat2 = train_labels_cat[val_count:,:]


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def build_model():
    model = Sequential()
    # add Convolutional layers
    model.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same',
                     input_shape=(image_height, image_width, num_channels)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))    
    model.add(Flatten())
    # Densely connected layers
    model.add(Dense(128, activation='relu'))
    # output layer
    model.add(Dense(num_classes, activation='softmax'))
    # compile with adam optimizer & categorical_crossentropy loss function
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_model()
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 3, 3, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 576)              

In [ ]:
results = model.fit(train_data2, train_labels_cat2, 
                    epochs=15, batch_size=64,
                    validation_data=(val_data, val_labels_cat))

Epoch 1/15
657/657 [==============================] - 63s 96ms/step - loss: 0.2117 - accuracy: 0.9361 - val_loss: 0.0591 - val_accuracy: 0.9822
Epoch 2/15
657/657 [==============================] - 63s 96ms/step - loss: 0.0550 - accuracy: 0.9831 - val_loss: 0.0783 - val_accuracy: 0.9758
Epoch 3/15
657/657 [==============================] - 64s 97ms/step - loss: 0.0401 - accuracy: 0.9875 - val_loss: 0.0418 - val_accuracy: 0.9862
Epoch 4/15
657/657 [==============================] - 63s 96ms/step - loss: 0.0317 - accuracy: 0.9895 - val_loss: 0.0362 - val_accuracy: 0.9883
Epoch 5/15
657/657 [==============================] - 63s 96ms/step - loss: 0.0250 - accuracy: 0.9919 - val_loss: 0.0425 - val_accuracy: 0.9869
Epoch 6/15
657/657 [==============================] - 64s 98ms/step - loss: 0.0193 - accuracy: 0.9940 - val_loss: 0.0357 - val_accuracy: 0.9893
Epoch 7/15
657/657 [==============================] - 64s 98ms/step - loss: 0.0157 - accuracy: 0.9948 - val_loss: 0.0364 - val_accuracy:

In [ ]:
test_loss, test_accuracy = model.evaluate(test_data, test_labels_cat, batch_size=64)
print('Test loss: %.4f accuracy: %.4f' % (test_loss, test_accuracy))

157/157 [==============================] - 4s 25ms/step - loss: 0.0366 - accuracy: 0.9915
Test loss: 0.0366 accuracy: 0.9915
